# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Jenny Sofía Guerrero Burbbano  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
 Configuración de Tribunales

Descripción del problema:

Se precisa configurar tribunales de evaluación para un grupo de 15 alumnos que desean presentar
su Trabajo Fin de Máster (TFM). fi Cada tribunal está compuesto por tres profesores, cada uno
desempeñando uno de los siguientes roles: Presidente, Secretario o Vocal. fi Los profesores han
indicado su disponibilidad horaria para participar en los tribunales de 15h a 21h durante la semana
del 15 al 19 de abril: Número de profesores : 10 Número de tribunales : 15 Disponibilidad/Roles :
https://bit.ly/41QWk8o - 1 indica que profesor tiene disponibilidad - 0 en caso contrario
#Modelo - ¿Como represento el espacio de soluciones?
El espacio de soluciones está compuesto por
todas las combinaciones posibles de tribunales teniendo en cuenta las restricciones. Cada solución
puede ser una lista de tribunales, donde cada tribunal es una tupla de tres profesores (Presidente,
Secretario, Vocal). - ¿Cual es la función objetivo? La función objetivo podría ser minimizar la varianza en la cantidad de tribunales a los que cada profesor es asignado. Buscamos una distribución
equitativa de la carga de trabajo entre todos los profesores. También podríamos tener una función
objetivo secundaria, como maximizar la diversidad en la composición de los tribunales, asegurándonos de que los profesores trabajen con diferentes colegas. - ¿Como implemento las restricciones?
Las restricciones se pueden implementar de la siguiente manera:
Restricción de disponibilidad: Cada profesor tiene ciertas horas en las que está disponible. Esta restricción se implementa asegurándose de que los profesores solo sean asignados a tribunales durante
su horario disponible. Restricción de unicidad: Un profesor no puede estar en dos tribunales al
mismo tiempo. Esto se implementa mediante la verificación de que cada profesor solo sea asignado
a un tribunal por franja horaria. Restricción de roles: Cada tribunal debe tener un Presidente, un
Secretario y un Vocal, y ningún profesor puede desempeñar más de un rol a la vez.






                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

In [7]:
import pandas as pd
import matplotlib as mtlp

# Function to read the CSV file containing professor roles and convert to a usable structure
def read_professor_roles(csv_file_path):
    df_roles = pd.read_csv(csv_file_path)
    # Convert the roles to a dictionary with professors as keys and their roles as values
    professor_roles = {row['Profesor']: row['ROL'].split(',') for _, row in df_roles.iterrows()}
    return professor_roles

# Corrected function to read the professor availability
def read_professor_availability_corrected(csv_file_path):
    df_availability = pd.read_csv(csv_file_path)
    # Set the 'Profesor' column as the index of the dataframe
    df_availability.set_index('Profesor', inplace=True)
    return df_availability

# Function to create a single tribunal given the availability and roles
def create_single_tribunal(professor_roles, professor_availability):
    # For each time slot, check if there are enough professors available to form a tribunal
    for column in professor_availability.columns:
        available_professors = professor_availability.index[professor_availability[column] == 1].tolist()
        # We need to find three professors with different roles who are available at the same time
        for president in available_professors:
            if 'P' in professor_roles[president]:
                for secretary in available_professors:
                    if secretary != president and 'S' in professor_roles[secretary]:
                        for vocal in available_professors:
                            if vocal != president and vocal != secretary and 'V' in professor_roles[vocal]:
                                # Mark the professors as unavailable for this time slot
                                professor_availability.at[president, column] = 0
                                professor_availability.at[secretary, column] = 0
                                professor_availability.at[vocal, column] = 0
                                # Return the tribunal with the assigned roles and time slot
                                return {'Fecha': column.split('/')[0], 'Hora': column.split('/')[1],
                                        'Presidente': president, 'Secretario': secretary, 'Vocal': vocal}
    # If we cannot form a tribunal, return None
    return None

# Read the professor roles and availabilities from the CSV files
professor_roles = read_professor_roles('profesores_roles.csv')
professor_availability = read_professor_availability_corrected('tribunales_horario_completo.csv')

# Create the list of tribunals
tribunal_list = []
for _ in range(15):  # We want to create 15 tribunals
    tribunal = create_single_tribunal(professor_roles, professor_availability)
    if tribunal is not None:
        tribunal_list.append(tribunal)

# Convert the tribunal list to a DataFrame
tribunals_df = pd.DataFrame(tribunal_list)
print(tribunals_df)

# Save the DataFrame to a CSV file
tribunals_csv_path = 'tribunals_final.csv'
tribunals_df.to_csv(tribunals_csv_path, index_label='Tribunal')




   Fecha Hora Presidente Secretario Vocal
0     15   15        QYV        HLC   MSB
1     15   15        PMQ        EBB   IOE
2     15   16        RRD        QYV   MSB
3     15   16        PMQ        QWF   EBB
4     15   17        RRD        QYV   LHL
5     15   17        PMQ        HLC   QWF
6     15   18        RRD        QYV   LHL
7     15   18        MSB        PMQ   QWF
8     15   18        IOE        EBB   IOA
9     15   19        PMQ        HLC   QWF
10    15   20        RRD        HLC   LHL
11    15   20        MSB        QWF   IOE
12    15   21        RRD        MSB   LHL
13    16   15        RRD        HLC   LHL
14    16   15        MSB        PMQ   QWF


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

Complejidad Computacional:

La complejidad computacional de este problema puede ser analizada en términos de la cantidad de profesores (P), la cantidad de franjas horarias disponibles (H) y el número de roles a asignar (R). En este caso, R es constante ya que siempre hay tres roles (Presidente, Secretario, Vocal).

El algoritmo que hemos implementado revisa cada franja horaria para cada profesor y verifica las combinaciones posibles de roles. La parte más intensiva del algoritmo es la generación de combinaciones de profesores para formar tribunales.

Para cada franja horaria, generamos todas las combinaciones posibles de 3 profesores de un conjunto de P profesores. El número de tales combinaciones es dado por la fórmula de combinación C(P, 3), que es igual a P! / (3!(P-3)!), donde "!" denota factorial.


Espacio de Soluciones:

El espacio de soluciones está compuesto por todas las combinaciones válidas de tribunales que se pueden formar teniendo en cuenta las restricciones de disponibilidad y roles. Si consideramos que hay P profesores y cada profesor puede ocupar uno de los 3 roles, el espacio de soluciones sin restricciones sería P^3 para cada tribunal, ya que cada uno de los 3 roles puede ser ocupado por cualquiera de los P profesores.

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

La programación de restricciones utiliza técnicas de backtracking y poda para descartar rápidamente grandes porciones del espacio de búsqueda que no pueden conducir a una solución válida, mejorando la eficiencia del proceso de búsqueda.
